In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import datetime as dt
import csv

hostip = "192.168.1.111"

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

if __name__ == '__main__':
    topic = 'Transaction'
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Exiting script due to connection error.')
        exit()

    log_file = 'data/LI-Small_Trans.csv'

    with open(log_file, 'r') as file:
        reader = csv.reader(file, delimiter=',')
        headers = next(reader) 

        records_to_publish = []

        for row in reader:
            record = {
                'Timestamp': dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'From_Bank': row[1] if len(row) > 1 else '',
                'Account': row[2] if len(row) > 2 else '',
                'To_Bank': row[3] if len(row) > 3 else '',
                'To_Account': row[4] if len(row) > 4 else '',
                'Amount_Received': row[5] if len(row) > 5 else '',
                'Receiving_Currency': row[6] if len(row) > 6 else '',
                'Amount_Paid': row[7] if len(row) > 7 else '',
                'Payment_Currency': row[8] if len(row) > 8 else '',
                'Payment_Format': row[9] if len(row) > 9 else '',
                'Is_Laundering': row[10] if len(row) > 10 else ''

            }

            records_to_publish.append(record)

            if len(records_to_publish) == 5:
                publish_message(producer, topic, records_to_publish)
                records_to_publish = []

            sleep(1)

        if records_to_publish:
            publish_message(producer, topic, records_to_publish)
            
